In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import keras
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras_pyramid_pooling_module import PyramidPoolingModule

In [ ]:
def plot_image_grid(data, figsize=(15, 15), cmap=None, cbar=True):
    """
    Plot the data as a grid of images.

    Args:
        data: the tensor of image data to plot in
        (M, N, H, W, C) format where M is the 
        height of the image grid, N is the width
        of the image grid, H is the height of the
        image, W is the width of the image, and C
        is the channel dimensionality of the image
        cmap: the color map to use for the data
        cbar: whether to include a color bar legend

    Returns:
        None

    """
    M, N = data.shape[0], data.shape[1]
    fig, ax = plt.subplots(nrows=M, ncols=N, sharex=True, sharey=True, figsize=figsize)
    for i in range(M):    
        for j in range(N): 
            idx = i + 1 + N * j
            im = ax[i, j].imshow(data[i, j], cmap=cmap)
            ax[i, j].axes.xaxis.set_major_locator(plt.NullLocator())
            ax[i, j].axes.yaxis.set_major_locator(plt.NullLocator())
    if cbar:
        cb_ax = fig.add_axes([1., 0.2, 0.02, 0.6])
        cbar = fig.colorbar(im, cax=cb_ax)

# Dataset

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
# normalize images into [0, 1]
X_train = X_train[..., None] / 255.0
X_test = X_test[..., None] / 255.0
# get the target size of the images and number of classes
TARGET_SIZE = X_train.shape[1:]
NUM_CLASSES = np.max(y_train) + 1
# convert discrete labels to one-hot vectors
y_train = np.eye(NUM_CLASSES)[y_train.flatten()]
y_test = np.eye(NUM_CLASSES)[y_test.flatten()]

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
plot_image_grid(X_train[:25].reshape(5, 5, 28, 28, 1), cbar=False, cmap='bone')

# Model

## Baseline

In [ ]:
input_layer = Input(TARGET_SIZE)
x = input_layer
x = Conv2D(64, (2, 2), activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Conv2D(64, (2, 2), activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Conv2D(64, (2, 2), activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(10, activation='softmax')(x)
model = Model(inputs=input_layer, outputs=x)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history1 = model.fit(X_train, y_train, 
    epochs=100, 
    batch_size=10, 
    callbacks=[keras.callbacks.EarlyStopping()],
    validation_split=0.3,
    shuffle=True,
)

## Pyramid Pooling Near Output

In [ ]:
input_layer = Input(TARGET_SIZE)
x = input_layer
x = Conv2D(64, (2, 2), activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Conv2D(64, (2, 2), activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Conv2D(64, (2, 2), activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = PyramidPoolingModule()(x)
x = Flatten()(x)
x = Dense(10, activation='softmax')(x)
model = Model(inputs=input_layer, outputs=x)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history2 = model.fit(X_train, y_train, 
    epochs=100, 
    batch_size=10, 
    callbacks=[keras.callbacks.EarlyStopping()],
    validation_split=0.3,
    shuffle=True,
)

## Pyramid Pooling Near Input

In [ ]:
input_layer = Input(TARGET_SIZE)
x = input_layer
x = PyramidPoolingModule()(x)
x = Conv2D(64, (2, 2), activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Conv2D(64, (2, 2), activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Conv2D(64, (2, 2), activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(10, activation='softmax')(x)
model = Model(inputs=input_layer, outputs=x)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history3 = model.fit(X_train, y_train, 
    epochs=100, 
    batch_size=10, 
    callbacks=[keras.callbacks.EarlyStopping()],
    validation_split=0.3,
    shuffle=True,
)